<a href="https://colab.research.google.com/github/JBlake26/Real-Time-ML/blob/main/Homework2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Problem 1.1**

In [1]:
import numpy as np 
import pandas as pd 
import torch
import torch.optim as optim
import torch.nn as nn

# Data Visualisation 
import matplotlib.pyplot as plt  
import seaborn as sns

In [2]:
housing = pd.DataFrame(pd.read_csv("/content/drive/MyDrive/ECGR 4106 - Real Time Machine Learning/Housing.csv")) 
housing.head() 

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [3]:
m = len(housing) 
m 

545

In [4]:
housing.shape 

(545, 13)

In [5]:
housing.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


In [6]:
housing.describe() 

,price,area,bedrooms,bathrooms,stories,parking
count,5.450000e+02,545.000000,545.000000,545.000000,545.000000,545.000000
mean,4.766729e+06,5150.541284,2.965138,1.286239,1.805505,0.693578
std,1.870440e+06,2170.141023,0.738064,0.502470,0.867492,0.861586
min,1.750000e+06,1650.000000,1.000000,1.000000,1.000000,0.000000
25%,3.430000e+06,3600.000000,2.000000,1.000000,1.000000,0.000000
50%,4.340000e+06,4600.000000,3.000000,1.000000,2.000000,0.000000
75%,5.740000e+06,6360.000000,3.000000,2.000000,2.000000,1.000000
max,1.330000e+07,16200.000000,6.000000,4.000000,4.000000,3.000000


In [7]:
# You can see that your dataset has many columns with values as 'Yes' or 'No'. 
# But in order to fit a regression line, we would need numerical values and not string.
# List of variables to map 
 
varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning'] 
 
# Defining the map function 
def binary_map(x): 
    return x.map({'yes': 1, "no": 0}) 
 
# Applying the function to the housing list 
housing[varlist] = housing[varlist].apply(binary_map) 
 
# Check the housing dataframe now 
housing.head() 

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,yes,furnished
1,12250000,8960,4,4,4,1,0,0,0,1,3,no,furnished
2,12250000,9960,3,2,2,1,0,1,0,0,2,yes,semi-furnished
3,12215000,7500,4,2,2,1,0,1,0,1,3,yes,furnished
4,11410000,7420,4,1,2,1,1,1,0,1,2,no,furnished


In [8]:
#Splitting the Data into Training and Testing Sets 
from sklearn.model_selection import train_test_split 
 
# We specify this so that the train and test data set always have the same rows, respec
np.random.seed(0) 
df_train, df_test = train_test_split(housing, train_size = 0.8, test_size = 0.2, random_state=42)
df_train.shape

(436, 13)

In [9]:
df_test.shape 

(109, 13)

In [10]:

num_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking','price'] 
df_Newtrain = df_train[num_vars] 
df_Newtest = df_test[num_vars] 
df_Newtrain.head() 

,area,bedrooms,bathrooms,stories,parking,price
46,6000,3,2,4,1,7525000
93,7200,3,2,1,3,6300000
335,3816,2,1,1,2,3920000
412,2610,3,1,2,0,3430000
471,3750,3,1,2,0,3010000


In [11]:
df_Newtrain.shape 

(436, 6)

In [12]:
# Here we can see that except for area, all the columns have small integer values.  
#So it is extremely important to rescale the variables so that they have a comparable s
#If we don't have comparable scales, then some of the coefficients as obtained by fitti
#This might become very annoying at the time of model evaluation.  
##So it is advised to use standardization or normalization so that the units of the coef
 
#As you know, there are two common ways of rescaling:
#1. Min-Max scaling 
#2. Standardisation (mean-0, sigma-1) 
 
import warnings 
warnings.filterwarnings('ignore') 
 
from sklearn.preprocessing import MinMaxScaler, StandardScaler 
 
# define standard scaler 
#scaler = StandardScaler() 
scaler = MinMaxScaler() 
df_Newtrain[num_vars] = scaler.fit_transform(df_Newtrain[num_vars])
df_Newtest[num_vars] = scaler.fit_transform(df_Newtest[num_vars])
df_Newtrain.head(20) 

,area,bedrooms,bathrooms,stories,parking,price
46,0.298969,0.4,0.333333,1.000000,0.333333,0.550000
93,0.381443,0.4,0.333333,0.000000,1.000000,0.433333
335,0.148866,0.2,0.000000,0.000000,0.666667,0.206667
412,0.065979,0.4,0.000000,0.333333,0.000000,0.160000
471,0.144330,0.4,0.000000,0.333333,0.000000,0.120000
237,0.230928,0.4,0.000000,0.333333,0.000000,0.273333
544,0.151203,0.4,0.000000,0.333333,0.000000,0.000000
470,0.235464,0.4,0.000000,0.000000,0.000000,0.120000
511,0.105155,0.4,0.000000,0.000000,0.000000,0.073333
361,0.300344,0.4,0.000000,0.000000,0.000000,0.186667


In [13]:
y_Newtrain = df_Newtrain.pop('price') 
x_Newtrain = df_Newtrain
y_Newtest = df_Newtest.pop('price')
x_Newtest = df_Newtest

In [14]:
y_Newtrain = torch.tensor(y_Newtrain.values).float()
y_Newtrain = y_Newtrain[:,None]
x_Newtrain = torch.tensor(x_Newtrain.values).float()
y_Newtest = torch.tensor(y_Newtest.values).float()
y_Newtest = y_Newtest[:,None]
x_Newtest = torch.tensor(x_Newtest.values).float()

In [15]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) # <1>
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [20]:
seq_model1 = nn.Sequential(
    nn.Linear(5,8),
    nn.Tanh(),
    nn.Linear(8, 5))
seq_model1

Sequential(
  (0): Linear(in_features=5, out_features=8, bias=True)
  (1): Tanh()
  (2): Linear(in_features=8, out_features=5, bias=True)
)

**Problem 1.2**

In [22]:
seq_model2 = nn.Sequential(
    nn.Linear(5,8),
    nn.Tanh(),
    nn.Linear(8,4),
    nn.Tanh(),
    nn.Linear(4,2),
    nn.Tanh(),
    nn.Linear(2, 5))
seq_model2

Sequential(
  (0): Linear(in_features=5, out_features=8, bias=True)
  (1): Tanh()
  (2): Linear(in_features=8, out_features=4, bias=True)
  (3): Tanh()
  (4): Linear(in_features=4, out_features=2, bias=True)
  (5): Tanh()
  (6): Linear(in_features=2, out_features=5, bias=True)
)

In [21]:
optimizer = optim.SGD(seq_model1.parameters(), lr=1e-3)
training_loop(200, optimizer, seq_model1, nn.MSELoss(), x_Newtrain, x_Newtest, y_Newtrain, y_Newtest)

Epoch 1, Training loss 0.2979, Validation loss 0.3020
Epoch 10, Training loss 0.2935, Validation loss 0.2977
Epoch 20, Training loss 0.2888, Validation loss 0.2930
Epoch 30, Training loss 0.2842, Validation loss 0.2884
Epoch 40, Training loss 0.2797, Validation loss 0.2839
Epoch 50, Training loss 0.2752, Validation loss 0.2795
Epoch 60, Training loss 0.2709, Validation loss 0.2751
Epoch 70, Training loss 0.2666, Validation loss 0.2709
Epoch 80, Training loss 0.2624, Validation loss 0.2668
Epoch 90, Training loss 0.2583, Validation loss 0.2627
Epoch 100, Training loss 0.2543, Validation loss 0.2587
Epoch 110, Training loss 0.2504, Validation loss 0.2548
Epoch 120, Training loss 0.2465, Validation loss 0.2509
Epoch 130, Training loss 0.2427, Validation loss 0.2472
Epoch 140, Training loss 0.2390, Validation loss 0.2435
Epoch 150, Training loss 0.2353, Validation loss 0.2399
Epoch 160, Training loss 0.2318, Validation loss 0.2363
Epoch 170, Training loss 0.2283, Validation loss 0.2328
Epo

In [23]:
optimizer = optim.SGD(seq_model2.parameters(), lr=1e-3)
training_loop(200, optimizer, seq_model2, nn.MSELoss(), x_Newtrain, x_Newtest, y_Newtrain, y_Newtest)

Epoch 1, Training loss 0.2499, Validation loss 0.2590
Epoch 10, Training loss 0.2482, Validation loss 0.2572
Epoch 20, Training loss 0.2463, Validation loss 0.2553
Epoch 30, Training loss 0.2443, Validation loss 0.2534
Epoch 40, Training loss 0.2424, Validation loss 0.2515
Epoch 50, Training loss 0.2406, Validation loss 0.2496
Epoch 60, Training loss 0.2387, Validation loss 0.2477
Epoch 70, Training loss 0.2368, Validation loss 0.2459
Epoch 80, Training loss 0.2350, Validation loss 0.2440
Epoch 90, Training loss 0.2332, Validation loss 0.2422
Epoch 100, Training loss 0.2314, Validation loss 0.2404
Epoch 110, Training loss 0.2296, Validation loss 0.2386
Epoch 120, Training loss 0.2278, Validation loss 0.2368
Epoch 130, Training loss 0.2261, Validation loss 0.2351
Epoch 140, Training loss 0.2244, Validation loss 0.2333
Epoch 150, Training loss 0.2226, Validation loss 0.2316
Epoch 160, Training loss 0.2209, Validation loss 0.2299
Epoch 170, Training loss 0.2192, Validation loss 0.2282
Epo